# Assignment NMT: Intro to NLP

The aim of this homework is to familiarize you with sequence-to-sequence language modeling, specifically using an encoder-decoder model. In this notebook, you are provided with pre-written code for a simple sequence-to-sequence model that already works and learns how to reverse short sequences of numbers.

If you run this whole jupyter notebook, it will learn to reverse short sequences of numbers. Although much of this code you will not be modifying, we recommend reading through it to get a sense of how the model and training works.

This starter code is based on [this tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) by Sean Robertson from the PyTorch website and the COMS course at Columbia University by Professor Kathy McKeown. 

### Overview

Your assignment is to:
 1. adapt this notebook to work on the English-Italian language pair from Tataoeba website
 2. Implement a beam search function
 

Write all your code **in this jupyter notebook**. Cells are provided where you should be implementing your code. 

You do not need to modify any code to train the model. You may modify the `trainIters` function, if you would like to improve how you track progress, or change parameters while training. For example, it can be useful to decrease the teacher-forcing ratio as training progresses.


I would recommend you run this notebook as it is, first. You should be able to run it with the dummy data provided without making ANY modifications (except the cell where the data are loaded). Then, start making your changes as requested.



In [ ]:
# You may modify this cell if you like

import random
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchtext

In [ ]:
# DO NO MODIFY

# this is useful for checking if your code is successfully using the GPU

mydevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mydevice

device(type='cpu')

In [ ]:
# DO NOT MODIFY

SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

MAX_LEN = 50

def len_filter(example):
    return len(example.src) <= MAX_LEN and len(example.tgt) <= MAX_LEN

### Load dummy number reversal dataset

In [ ]:
#note that my files were stored in google drive and I was using google colab to run this notebook
#you can change this cell to provide local path to load your training and dev data

from google.colab import drive
drive.mount('/content/drive')

train_path = '/content/drive/My Drive/COURSES/2020/data/train/data.txt'
dev_path = '/content/drive/My Drive/COURSES/2020/data/dev/data.txt'


Mounted at /content/drive


In [ ]:
# DO NOT MODIFY


src = torchtext.data.Field(
    batch_first=True, 
    include_lengths=True
    )
tgt = torchtext.data.Field(
    batch_first=True, 
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN]
    )

data_train = torchtext.data.TabularDataset(
        path=train_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
data_dev = torchtext.data.TabularDataset(
        path=dev_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )

### 1. Load the data (10 points)

Load in the en-it data from http://www.manythings.org/anki/ita-eng.zip

 similar to how the dummy number reversal dataset is loaded above. That is, use the same `torchtext.data.Field` and `torchtext.data.TabularDataset` classes.

In [ ]:
# WRITE YOUR CODE FOR LOADING THE ITALIAN<->ENGLISH DATA HERE
#TODO 
#create 6 files, English|Italian data for train|dev|test, one sentence per line.
#you can ignore this cell when you are running the code the first time through on the dummy reversal dataset

Have a look at the vocab and some example data points.

*If you have loaded in the data correctly, the code in the cell below should work without any modification.*

In [ ]:
# You may modify this cell if you like

src.build_vocab(data_train, max_size=50000)
tgt.build_vocab(data_train, max_size=50000)
input_vocab = src.vocab
output_vocab = tgt.vocab

print('20 tokens from input vocab:\n', list(input_vocab.stoi.keys())[:20])
print('\n20 tokens from output vocab:\n', list(output_vocab.stoi.keys())[:20])

print('\nnum training examples:', len(data_train.examples))

item = random.choice(data_train.examples)
print('\nexample train data:')
print('src:\n', item.src)
print('tgt:\n', item.tgt)

#showing output below for toy dataset

20 tokens from input vocab:
 ['<unk>', '<pad>', '2', '0', '7', '9', '6', '4', '3', '8', '1', '5']

20 tokens from output vocab:
 ['<unk>', '<pad>', '<eos>', '<sos>', '2', '0', '7', '9', '6', '4', '3', '8', '1', '5']

num training examples: 10000

example train data:
src:
 ['3', '6', '5']
tgt:
 ['<sos>', '5', '6', '3', '<eos>']


### Model definition and training functions

In [ ]:
# DO NOT MODIFY

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, myinput, hidden):
        embedded = self.embedding(myinput).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

In [ ]:
# DO NOT MODIFY

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
          max_length=MAX_LEN, teacher_forcing_ratio=0.5):
    
    # get an initial hidden state for the encoder
    encoder_hidden = encoder.initHidden()

    # zero the gradients of the optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # get the seq lengths, used for iterating through encoder/decoder
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # create empty tensor to fill with encoder outputs
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

    # create a variable for loss
    loss = 0
    
    # pass the inputs through the encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # create a start-of-sequence tensor for the decoder
    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

    # set the decoder hidden state to the final encoder hidden state
    decoder_hidden = encoder_hidden

    # decide if we will use teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input
                
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        
        if use_teacher_forcing:
            decoder_input = target_tensor[di]
        
        if decoder_input.item() == output_vocab.stoi[EOS_TOKEN]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
# You may modify this cell

def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01, teacher_forcing_ratio=0.5):
    print(f'Running {n_iters} epochs...')
    print_loss_total = 0
    print_loss_epoch = 0

    encoder_optim = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optim = optim.SGD(decoder.parameters(), lr=learning_rate)

    # note batch size of 1, just for simplicity
    # DO NOT INCREASE THE BATCH SIZE
    batch_iterator = torchtext.data.Iterator(
        dataset=data_train, batch_size=1,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=mydevice, repeat=False)
    

    criterion = nn.NLLLoss()

    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        start = time.time()
        for batch in batch_generator:
            step += 1
            
            # get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            target_tensor = getattr(batch, 'tgt')
            
            # this is because we're not actually using the batches.
            # batch size is 1 and this just selects that first one
            input_tensor = input_tensor[0]
            target_tensor = target_tensor[0]

            loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optim, decoder_optim, criterion, teacher_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            print_loss_epoch += loss
            

            if step % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                t = (time.time() - start) / 60
                print(f'step: {step}\t avg loss: {print_loss_avg:.2f}\t time for {print_every} steps: {t:.2f} min')
                start = time.time()
        
        print_loss_avg = print_loss_epoch / step
        print_loss_epoch = 0
        print(f'End of epoch {e}, avg loss {print_loss_avg:.2f}')  

###  Create and train a model

In [ ]:
# You may modify this cell

hidden_size = 128
encoder1 = EncoderRNN(len(input_vocab), hidden_size).to(mydevice)
decoder1 = DecoderRNN(hidden_size, len(output_vocab)).to(mydevice)

Here are some guidelines for how much training to expect. Note that these *guidelines*; they are not exact.

Only 1 epoch is needed for the number reversal dataset. This produces near-perfect results, and should take less than 5 minutes to run on a CPU.



In [ ]:
# You may modify this cell
# but be sure that it prints some indication of how training is progressing

trainIters(encoder1, decoder1, 1, print_every=1000, teacher_forcing_ratio=0.75)

Running 1 epochs...
step: 1000	 avg loss: 1.59	 time for 1000 steps: 0.17 min
step: 2000	 avg loss: 0.97	 time for 1000 steps: 0.16 min
step: 3000	 avg loss: 0.69	 time for 1000 steps: 0.16 min
step: 4000	 avg loss: 0.64	 time for 1000 steps: 0.17 min
step: 5000	 avg loss: 0.59	 time for 1000 steps: 0.17 min
step: 6000	 avg loss: 0.53	 time for 1000 steps: 0.17 min
step: 7000	 avg loss: 0.53	 time for 1000 steps: 0.17 min
step: 8000	 avg loss: 0.48	 time for 1000 steps: 0.16 min
step: 9000	 avg loss: 0.47	 time for 1000 steps: 0.16 min
step: 10000	 avg loss: 0.46	 time for 1000 steps: 0.17 min
End of epoch 0, avg loss 0.70


In [ ]:
# DO NOT MODIFY

def evaluate(encoder, decoder, sentence, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            next_word = output_vocab.itos[topi.item()]
            decoded_words.append(next_word)
            if next_word == EOS_TOKEN:
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

Have a look at some generated sequences! This is the fun part.

In [ ]:
# You may modify this cell

# This selects 5 random datapoints from the training data and shows the generated sequence

for i in range(5):
    item = random.choice(data_train.examples)
    seq = item.src
    print(seq)
    words = evaluate(encoder1, decoder1, seq)
    print(' '.join(words))
    print()

['6', '3', '5', '4', '8', '9']
<sos> 9 8 4 5 3 6 <eos>

['5', '6', '6', '9']
<sos> 9 6 5 6 6 <eos>

['5', '8', '3', '2']
<sos> 2 3 8 5 <eos>

['0', '1', '6', '9']
<sos> 9 1 6 0 <eos>

['2', '8', '7', '9', '1']
<sos> 1 9 7 8 2 <eos>



### Implement beam search (10 points for ITCS 4111 students, 15 points for the ITCS 5111/DSBA 6010 students)

We provide an evaluation function that performs greedy decoding on any input sequence provided. 

(ITCS 4111 students) You must write a new  function that implements beam search for a beam size = 2. 

(ITCS 5111/DSBA 6010 students) You must write a new  function that implements beam search for an arbitrary beam size. Let the beam size be an input parameter to your function. 

In greedy decoding, at each decoding step the most likely word is selected, resulting in one decoded sequence output. 

In beam search, at each decoding step the top k most
likely sequences are selected. Each of these k sequences is then used to generate the next step; the top k next words per sequence are considered (for a total of k ∗ k sequences)
and the top k sequences are selected to take to the next decoding step. At the end, you have k decoded sequence outputs.


In [ ]:
# WRITE YOUR CODE FOR THE BEAM SEARCH HERE


#there are several implementation of beam search on the web. You can adapt those for this assignment. 
#Please provide proper credit to the original source if you are doing so
#one example is here: https://github.com/budzianowski/PyTorch-Beam-Search-Decoding/blob/master/decode_beam.py


#you may need to add a helper function to see if your implementation of beam search works
#add that helper function below

